TODO:
- [ ] Set up the tools
- [ ] set up system prompt
- [ ] implement check token warning

In [2]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.tools.render import format_tool_to_openai_function
from langgraph.prebuilt import ToolExecutor

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage

from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

from langgraph.graph import StateGraph, END

In [4]:
# Define OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Define Tools

# Append to Context

# Replace context

# Search conversation history

# Search conversation history based on date

# Write to secondary memory

# Read from secondary memory

# Define functions to be binded to model
functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
# Define Prompt
system_prompt = ("")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# Define model and agent chain
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

mem_agent_chain = (
    prompt
    | model.bind_functions(functions)
    | JsonOutputFunctionsParser()
)

In [11]:
# Define Agent State
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    return_to_agent: bool

In [ ]:
# Define the function that determines whether to call function or go to checks
def should_call_tool(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        state["return_to_agent"] = false
        return "check token warning"
    # Otherwise if there is, we continue
    else:
        state["return_to_agent"] = true
        return "call tool"

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

# Define function to check and handle token warning
def check_token_warning(state):
    # If token warning, add message

def should_return_to_agent(state):
    if state["return to agent"] == true
        return "return to agent"
    else
        return "return to user"

In [ ]:
# Define a new graph
graph = StateGraph(AgentState)

# Define Nodes
graph.add_node("mem agent",  mem_agent_chain)
graph.add_node("read/write memory", call_tool)
graph.add_node("check token warning", check_token_warning)

# Set entry point
graph.set_entry_point("mem agent")

# Add conditional edge from agent to: read/write mem OR perform checks
graph.add_conditional_edges("agent", should_call_tool, {"call tool": "read/write memory", "check token warning": "check token warning"})

# Add edge from tool call to check token warning
graph.add_edge("read/write memory", "check token warning")

# Add conditional edge from perform checks to: Agent OR END
graph.add_conditional_edges("check_token_warning", should_return_to_agent, {"return to agent": "agent", "return to user": END) 